In [1]:
import sys
import os
import hydra
sys.path.append(os.path.abspath("/e2ws/exercises/corrdiff"))
import generate

# read config with hydra, clear in case hydra has been initialised before
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize(version_base="1.3", config_path='./exercises/corrdiff/conf')
cfg = hydra.compose(config_name='config_generate')
generate.main(cfg)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/sampler.py:64: UserWarning: `data_source` argument is not used and will be removed in 2.2.0.You may still have custom implementation that utilizes it.
  warnings.warn("`data_source` argument is not used and will be removed in 2.2.0."
/usr/local/lib/python3.10/dist-packages/modulus/distributed/manager.py:285: UserWarning: Distributed manager is already intialized
  warn("Distributed manager is already intialized")
[13:24:53 - generate - INFO] Patch-based generation disabled
[13:24:53 - generate - INFO] torch.__version__: 2.2.0a0+81ea7a4
[13:24:53 - generate - INFO] Loading residual network from "/e2ws/exercises/corrdiff/checkpoints/diffusion.mdlus"...
[13:24:54 - generate - INFO] Loading network from "/e2ws/exercises/corrdiff/checkpoints/regression.mdlus"...
[13:24:56 - generate - INFO] Generating images...
[13:24:56 - generate - INFO] starting index: 0
[13:24:56 - generate - INFO] seeds: [0, 1, 2, 3, 4, 5, 6, 7]
100%|████████████

In [2]:
import xarray

def open_results(path):
    root = xarray.open_dataset(path)
    pred = (xarray.open_dataset(path, group="prediction")
            .merge(root).set_coords(["lat", "lon"]))
    truth = (xarray.open_dataset(path, group="truth")
             .merge(root).set_coords(["lat", "lon"]))
    return pred, truth

pred, truth = open_results(f"{cfg['image_outdir']}_0.nc")
pred

<xarray.Dataset>
Dimensions:                     (ensemble: 8, time: 3, y: 448, x: 448)
Coordinates:
  * time                        (time) datetime64[ns] 2021-09-12 ... 2021-09-...
    lat                         (y, x) float32 ...
    lon                         (y, x) float32 ...
Dimensions without coordinates: ensemble, y, x
Data variables:
    maximum_radar_reflectivity  (ensemble, time, y, x) float32 ...
    temperature_2m              (ensemble, time, y, x) float32 ...
    eastward_wind_10m           (ensemble, time, y, x) float32 ...
    northward_wind_10m          (ensemble, time, y, x) float32 ...

In [10]:
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
# import matplotlib.animation as animation
# from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
# from numpy import datetime_as_string

# var = 'maximum_radar_reflectivity'

# n_samples = 4
# n_samples = min(n_samples, cfg["seed_batch_size"])
# output_dir = './outputs'

# # concatenate truth data and ensemble mean as an "ensemble" member for easy
# truth_expanded = truth.assign_coords(ensemble="truth").expand_dims("ensemble")
# ens_mean = (
#     pred.mean("ensemble")
#     .assign_coords(ensemble="ensemble_mean")
#     .expand_dims("ensemble")
# )
# # add [0, 1, 2, ...] to ensemble dim
# pred["ensemble"] = [str(i) for i in range(pred.sizes["ensemble"])]
# merged = xarray.concat([truth_expanded, ens_mean, pred], dim="ensemble")
# projection=ccrs.PlateCarree()

# # define plots
# def make_figure():
#     title = ['truth', 'mean']
#     # fig = plt.figure(figsize=(11,5))
#     fig.suptitle(f"{var} at {datetime_as_string(merged.time[0], unit='s')}", fontsize=18)
#     fig.set_tight_layout(True)
#     lon_formatter = LongitudeFormatter(zero_direction_label=False)
#     lat_formatter = LatitudeFormatter()
#     # ax = fig.add_subplot(1, 4, mem+1)


#     for mem in range(3):
#         ax = fig.add_subplot(1, 4, mem+1, projection=projection)
#         ax.add_feature(cfeature.COASTLINE,lw=.5)
#         ax.add_feature(cfeature.RIVERS,lw=.5)
#         ax.add_feature(cfeature.BORDERS, linewidth=0.6, edgecolor='dimgray')
#         ax.xaxis.set_major_formatter(lon_formatter)
#         ax.yaxis.set_major_formatter(lat_formatter)

#         if mem==2:
#             continue

#         plot_ds = merged[var][mem, 0, :, :]
#         pc = ax.pcolormesh(merged.lon, merged.lat, plot_ds, transform=projection,
#                         cmap='plasma', vmin=0, vmax=100)
#         ax.set_title(title[mem])

#     axx = fig.add_subplot(1, 4, 4)
#     cbar = fig.colorbar(pc, extend='both', shrink=0.6, ax=axx)
#     cbar.set_label('wind speed [m/s]', fontsize=12)

#     return fig, ax

# # plot the variables
# def make_frame(mem):
#     # 2 is for the esemble and truth
#     # merged[v][: n_samples + 2, :].plot(row="time", col="ensemble")
#     plot_ds = merged[var][mem+2, 0, :, :]
#     pc = ax.pcolormesh(merged.lon, merged.lat, plot_ds, transform=projection,
#                        cmap='plasma', vmin=0, vmax=100)
#     # if mem == -1:
#     #     cbar = fig.colorbar(pc, extend='both', shrink=0.6, ax=ax)
#     #     cbar.set_label('wind speed [m/s]', fontsize=12)
#     ax.set_title(f'ensemble member {mem+1} of {n_samples}')
#     return pc

# def animate(frame):
#     return make_frame(frame)

# def first_frame():
#     return make_frame(-1)

# %matplotlib inline
# plt.rcParams["animation.html"] = "jshtml"
# fig, ax = make_figure()
# ani = animation.FuncAnimation(fig,
#                               animate,
#                               n_samples,
#                               init_func=first_frame,
#                               blit=False,
#                               repeat=False,
#                               interval=.1)
# plt.close('all')
# ani

In [11]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.animation as animation
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from numpy import datetime_as_string

var = 'maximum_radar_reflectivity'
time_idx = 0

n_samples = 4
n_samples = min(n_samples, cfg["seed_batch_size"])
output_dir = './outputs'

# concatenate truth data and ensemble mean as an "ensemble" member for easy
truth_expanded = truth.assign_coords(ensemble="truth").expand_dims("ensemble")
ens_mean = (
    pred.mean("ensemble")
    .assign_coords(ensemble="ensemble_mean")
    .expand_dims("ensemble")
)
# add [0, 1, 2, ...] to ensemble dim
pred["ensemble"] = [str(i) for i in range(pred.sizes["ensemble"])]
merged = xarray.concat([truth_expanded, ens_mean, pred], dim="ensemble")
projection=ccrs.PlateCarree()

# define plots
def make_figure():
    title = ['truth', 'mean']
    fig, ax = plt.subplots(1, 3, figsize=(11,5), subplot_kw={'projection': ccrs.PlateCarree()})
    fig.suptitle(f"{var} at {datetime_as_string(merged.time[time_idx], unit='s')}", fontsize=18)
    # fig.set_tight_layout(True)
    lon_formatter = LongitudeFormatter(zero_direction_label=False)
    lat_formatter = LatitudeFormatter()


    for mem in range(3):
        # ax[mem] = fig.add_subplot(1, 4, mem+1, projection=projection)
        ax[mem].add_feature(cfeature.COASTLINE,lw=.5)
        ax[mem].add_feature(cfeature.RIVERS,lw=.5)
        ax[mem].add_feature(cfeature.BORDERS, linewidth=0.6, edgecolor='dimgray')
        ax[mem].xaxis.set_major_formatter(lon_formatter)
        ax[mem].yaxis.set_major_formatter(lat_formatter)

        if mem==2:
            continue

        plot_ds = merged[var][mem, 0, :, :]
        pc = ax[mem].pcolormesh(merged.lon, merged.lat, plot_ds, transform=projection,
                        cmap='plasma', vmin=0, vmax=100)
        ax[mem].set_title(title[mem])

    cbar = fig.colorbar(pc, extend='both', shrink=0.6, ax=ax, location='bottom')
    cbar.set_label(var, fontsize=12)

    return fig, ax

# plot the variables
def make_frame(mem):
    # 2 is for the esemble and truth
    plot_ds = merged[var][mem+2, 0, :, :]
    pc = ax[2].pcolormesh(merged.lon, merged.lat, plot_ds, transform=projection,
                       cmap='plasma', vmin=0, vmax=100)
    ax[2].set_title(f'ensemble member {mem+1} of {n_samples}')
    return pc

def animate(frame):
    return make_frame(frame)

def first_frame():
    return make_frame(-1)

%matplotlib inline
plt.rcParams["animation.html"] = "jshtml"
fig, ax = make_figure()
ani = animation.FuncAnimation(fig,
                              animate,
                              n_samples,
                              init_func=first_frame,
                              blit=False,
                              repeat=False,
                              interval=.1)
plt.close('all')
ani

In [12]:
merged[var][:, time]

<xarray.DataArray 'maximum_radar_reflectivity' (ensemble: 10, time: 3, y: 448,
                                                x: 448)>
array([[[[-2.38418579e-07,  2.38418579e-07,  2.38418579e-07, ...,
           2.38418579e-07,  2.38418579e-07, -2.38418579e-07],
         [ 0.00000000e+00,  2.38418579e-07,  2.38418579e-07, ...,
           2.38418579e-07,  2.38418579e-07,  0.00000000e+00],
         [ 2.38418579e-07,  2.38418579e-07,  2.38418579e-07, ...,
          -3.70359421e-03,  2.32195854e-03,  3.55434418e-03],
         ...,
         [ 0.00000000e+00,  2.38418579e-07,  2.38418579e-07, ...,
          -3.61251831e-03,  1.47271156e-03,  2.25400925e-03],
         [ 2.38418579e-07,  2.38418579e-07,  2.38418579e-07, ...,
           1.19614601e-03,  2.38418579e-07,  2.38418579e-07],
         [ 0.00000000e+00,  2.38418579e-07,  2.38418579e-07, ...,
           1.83081627e-03,  2.38418579e-07,  0.00000000e+00]],

        [[-2.38418579e-07,  2.38418579e-07,  2.38418579e-07, ...,
           2.38418579e-07,  2.38418579e-07, -2.38418579e-07],
         [ 0.00000000e+00,  2.38418579e-07,  2.38418579e-07, ...,
           2.38418579e-07,  2.38418579e-07,  0.00000000e+00],
         [ 2.38418579e-07,  2.38418579e-07,  2.38418579e-07, ...,
           2.38418579e-07,  2.38418579e-07,  2.38418579e-07],
...
          -4.51385498e+00, -4.14620018e+00, -7.97703314e+00],
         [ 2.59579635e+00,  1.85743558e+00,  4.73036170e-01, ...,
          -8.86951637e+00, -8.84059620e+00, -7.84873438e+00],
         [-6.16647243e-01, -5.35497427e+00, -5.22676802e+00, ...,
          -1.40489635e+01, -1.88331585e+01, -1.68949165e+01]],

        [[-2.45182443e+00,  2.72623825e+00, -1.04592061e+00, ...,
          -2.15192270e+00, -2.64042139e+00, -3.66882920e+00],
         [ 1.38278234e+00,  6.35161042e-01, -1.68572402e+00, ...,
          -2.81350923e+00, -8.04729748e+00, -4.05335283e+00],
         [-1.25813699e+00, -4.79379654e-01, -8.69111359e-01, ...,
          -3.46698254e-01, -3.71804619e+00,  7.94860065e-01],
         ...,
         [-3.62909222e+00, -7.95027971e-01,  3.10907140e-02, ...,
          -2.69184685e+00, -2.17016459e+00, -5.79936218e+00],
         [ 2.88439274e+00,  2.12383747e+00,  8.04246187e-01, ...,
          -6.53699446e+00, -6.37830448e+00, -5.72398996e+00],
         [-2.26987049e-01, -5.05083084e+00, -4.96324539e+00, ...,
          -1.11802578e+01, -1.61823654e+01, -1.48228703e+01]]]],
      dtype=float32)
Coordinates:
  * time      (time) datetime64[ns] 2021-09-12 ... 2021-09-12T12:00:00
    lat       (y, x) float32 19.55 19.55 19.55 19.55 ... 27.81 27.81 27.81 27.81
    lon       (y, x) float32 116.4 116.4 116.4 116.4 ... 125.5 125.5 125.5 125.5
  * ensemble  (ensemble) <U13 'truth' 'ensemble_mean' '0' '1' ... '5' '6' '7'
Dimensions without coordinates: y, x